In [1]:
import numpy as np
import pandas as pd
from sunpy.physics.differential_rotation import rot_hpc
from astropy import units as u
from shapely.geometry import Polygon, Point
from shapely import wkt
import os
import datetime
from sunpy.time import parse_time

In [2]:
def associate_sigmoid(inputFile_fl, inputFile_sigmoid, spatial_sep, output2file=False, out_file = None):
    #import a record of flare events as a DataFrame
    flare_set = pd.read_csv(inputFile_fl, delimiter = ',', header = 0)
    #import a record of sigmoid events as a DataFrame
    sigmoid_set = pd.read_csv(inputFile_sigmoid, delimiter = ',', header = 0)
    #set solar radius
    r = 966
    #how many flare events working with
    length = flare_set.shape[0]
    #list of zeroes with length of number of flare events
    zeroes = [0 for i in range(length)]
    #list of nulls with length of number of flare events
    nones = [None for i in range(length)]
    #create columns (filled with zeroes) for tracking associated events
    flare_set.loc[:, 'associated_sigmoid'] = nones
    flare_set.loc[:, 'num_associated_sigmoid'] = zeroes
    flare_set.loc[:, 'dist_sigmoid'] = zeroes

    sigmoid_set = sigmoid_set.rename(columns={'event_starttime':'starttime_sigmoid', 'event_endtime':'endtime_sigmoid',
                                              'hpc_bbox':'hpc_bbox_sigmoid', 'ar_noaanum': 'sigmoid_noaanum',
                                             'search_channelid': 'search_channelid_sigmoid'})
    sigmoid_keywords = ['starttime_sigmoid', 'endtime_sigmoid','hpc_bbox_sigmoid', 'sigmoid_noaanum', 
                        'search_channelid_sigmoid']
   
    #create columns filled with Nones for each sigmoid keyword
    for elem in sigmoid_keywords:
         flare_set.loc[:, elem] = nones
    #convert start and end times to datetime objects
    flare_set['event_starttime'] = map(parse_time, flare_set['event_starttime'])
    flare_set['event_endtime'] = map(parse_time, flare_set['event_endtime'])
    sigmoid_set['starttime_sigmoid'] = map(parse_time, sigmoid_set['starttime_sigmoid'])
    sigmoid_set['endtime_sigmoid'] = map(parse_time, sigmoid_set['endtime_sigmoid'])

    #set positional row index 
    i = -1

    for annoying_wrong_obj_type in flare_set['event_starttime']:
        i += 1
        #print which flare event function is currently processing, so the user has an idea of how much longer
        #program will need to run
        start = flare_set['event_starttime'].values[i]
        end = flare_set['event_endtime'].values[i]
        if (i+1)%100 == 0:
            print '%d / %d events' %((i+1), length)
        #begin eliminating sigmoid events based on temporal parameters
        sigmoid_search = sigmoid_set.ix[sigmoid_set['starttime_sigmoid']<=start]
        sigmoid_search = sigmoid_search.ix[sigmoid_search['endtime_sigmoid']>=end]
        num_associated_sigmoid=0
        #as long as the temporal search does not eliminate all possible related sigmoid events, proceed
        if sigmoid_search.empty==False:
                fl_point = Point((flare_set['hpc_x'].values[i], flare_set['hpc_y'].values[i]))
                min_s = spatial_sep
                event_index = None
                found_sigmoid = False
                for j in range(sigmoid_search.shape[0]):
                    #create a Shapely polygon object from sigmoid's location
                    sigmoid_poly = wkt.loads(sigmoid_search['hpc_bbox_sigmoid'].values[0])
                    #calculate the minimum 2D distance between the sigmoid's polygon and the flare's mean coordinates
                    chord = fl_point.distance(sigmoid_poly)
                    #calculate the minimum 3D distance along the sun's curved surface between the flare and sigmoid
                    #assumes the same radius for all events
                    s = r*np.arcsin(chord/(2*r))
                    #determine whether the spatial distance between flare and sigmoid meets the set parameter
                    if s <= spatial_sep:
                        #have found an associated sigmoid
                        found_sigmoid = True
                        num_associated_sigmoid+=1 
                        if s <= min_s: 
                            min_s = s
                            event_index = j
                if found_sigmoid:
                    flare_set['associated_sigmoid'].values[i] = sigmoid_search['SOL_standard'].values[event_index]
                    flare_set['num_associated_sigmoid'].values[i] = num_associated_sigmoid
                    flare_set['dist_sigmoid'].values[i] = min_s
                    for elem in sigmoid_keywords:
                        flare_set.loc[i, elem] = sigmoid_search[elem].values[event_index]
            
    #create boolean var to easily determine whether flare associated with an AR
    k = 0
    is_sigmoid = [0 for i in range(flare_set.shape[0])]
    for elem in flare_set['associated_sigmoid']:
        if elem!=None:
            is_sigmoid[k] = 1
        k+=1
    flare_set.loc[:, 'is_sigmoid'] = is_sigmoid
    
    #write dataframe to a csv file depending on initial parameters
    if output2file:
        if out_file == None:
            #create a generic name for file based on search parameters if no file name specified
             out_file = inputFile_fl[0:-4]+'_with_sigmoid.csv'
        #import which keywords to keep for outported data
        flare_keywords = list(np.genfromtxt('keywords_flare_with_goes_with_ar.csv', delimiter=',', dtype=str))
        #add to these keywords descriptors of associated sigmoid
        flare_keywords.extend(['is_sigmoid','associated_sigmoid', 'num_associated_sigmoid', 'dist_sigmoid'])
        flare_keywords.extend(sigmoid_keywords)
        #write to csv
        flare_set.to_csv(path_or_buf=out_file, columns = flare_keywords, index = False)
        
    return flare_set
    

In [3]:
# associate_sigmoid(inputFile_fl, inputFile_sigmoid, spatial_sep, output2file=False, out_file = None)
associate_sigmoid('flare_dataset_cleaned_30min_100arcsec_with_GOES_with_ar.csv', 'raw_sg.csv', 100, output2file=True,
                  out_file = None)

/usr/local/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2831: DtypeWarning: Columns (37,51,60,64,66,68,103,120) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


25 / 40293 events
50 / 40293 events
75 / 40293 events
100 / 40293 events
125 / 40293 events
150 / 40293 events
175 / 40293 events
200 / 40293 events
225 / 40293 events
250 / 40293 events
275 / 40293 events
300 / 40293 events
325 / 40293 events
350 / 40293 events
375 / 40293 events
400 / 40293 events
425 / 40293 events
450 / 40293 events
475 / 40293 events
500 / 40293 events
525 / 40293 events
550 / 40293 events
575 / 40293 events
600 / 40293 events
625 / 40293 events
650 / 40293 events
675 / 40293 events
700 / 40293 events
725 / 40293 events
750 / 40293 events
775 / 40293 events
800 / 40293 events
825 / 40293 events
850 / 40293 events
875 / 40293 events
900 / 40293 events
925 / 40293 events
950 / 40293 events
975 / 40293 events
1000 / 40293 events
1025 / 40293 events
1050 / 40293 events
1075 / 40293 events
1100 / 40293 events
1125 / 40293 events
1150 / 40293 events
1175 / 40293 events
1200 / 40293 events
1225 / 40293 events
1250 / 40293 events
1275 / 40293 events
1300 / 40293 events
13

,SOL_standard,event_starttime,event_endtime,event_peaktime,hpc_bbox,hpc_coord,hpc_radius,hpc_x,hpc_y,is_associated_fl,...,ar_hpc_bbox,associated_sigmoid,num_associated_sigmoid,dist_sigmoid,starttime_sigmoid,endtime_sigmoid,hpc_bbox_sigmoid,sigmoid_noaanum,search_channelid_sigmoid,is_sigmoid
0,SOL2010-06-11T20:27:24L105C069,2010-06-11 20:27:24,2010-06-11 21:02:48,2010-06-11T20:37:36,"POLYGON((614.4 307.2,691.2 307.2,691.2 384,614...",POINT(652.8 345.6),738.638748,652.8,345.6,1,...,"POLYGON((533.2764 333.4344,651.834 334.4202,62...",None,0,0,None,None,None,None,None,0
1,SOL2010-06-11T20:27:26L105C069,2010-06-11 20:27:26,2010-06-12 14:31:26,2010-06-12T01:00:14,"POLYGON((614.4 307.2,768 307.2,768 460.8,614.4...",POINT(652.8 345.6),738.638748,652.8,345.6,1,...,"POLYGON((533.2764 333.4344,651.834 334.4202,62...",None,0,0,None,None,None,None,None,0
2,SOL2010-06-12T05:20:39L100C069,2010-06-12 05:20:39,2010-06-12 05:27:03,2010-06-12T05:21:39,"POLYGON((614.4 307.2,691.2 307.2,691.2 384,614...",POINT(652.8 345.6),738.638748,652.8,345.6,1,...,"POLYGON((578.2764 324.369,713.37 325.8066,679....",None,0,0,None,None,None,None,None,0
3,SOL2010-06-12T09:04:03L098C069,2010-06-12 09:04:03,2010-06-12 09:29:27,2010-06-12T09:14:27,"POLYGON((614.4 307.2,768 307.2,768 460.8,614.4...",POINT(652.8 345.6),738.638748,652.8,345.6,1,...,"POLYGON((600.348 324.2352,736.116 325.854,699....",None,0,0,None,None,None,None,None,0
4,SOL2010-06-13T05:35:35L128C115,2010-06-13 05:35:35,2010-06-13 05:50:47,2010-06-13T05:40:59,"POLYGON((844.8 -460.8,921.6 -460.8,921.6 -307....",POINT(883.2 -422.4),979.011747,883.2,-422.4,1,...,NaN,None,0,0,None,None,None,None,None,0
5,SOL2010-06-13T05:54:48L105C069,2010-06-13 05:54:48,2010-06-13 05:58:24,2010-06-13T05:56:12,"POLYGON((768 307.2,844.8 307.2,844.8 384,768 3...",POINT(806.4 345.6),877.337062,806.4,345.6,1,...,"POLYGON((707.388 320.826,829.266 323.4732,787....",None,0,0,None,None,None,None,None,0
6,SOL2010-06-13T07:05:36L110C116,2010-06-13 07:05:36,2010-06-13 07:16:00,2010-06-13T07:08:48,"POLYGON((768 -460.8,844.8 -460.8,844.8 -384,76...",POINT(806.4 -422.4),910.331105,806.4,-422.4,1,...,NaN,None,0,0,None,None,None,None,None,0
7,SOL2010-06-13T07:33:00L097C064,2010-06-13 07:33:00,2010-06-13 07:38:24,2010-06-13T07:34:48,"POLYGON((691.2 307.2,768 307.2,768 460.8,691.2...",POINT(729.6 422.4),843.052739,729.6,422.4,1,...,"POLYGON((707.388 320.826,829.266 323.4732,787....",None,0,0,None,None,None,None,None,0
8,SOL2010-06-13T08:09:39L104C069,2010-06-13 08:09:39,2010-06-13 08:17:51,2010-06-13T08:13:03,"POLYGON((768 307.2,844.8 307.2,844.8 384,768 3...",POINT(806.4 345.6),877.337062,806.4,345.6,1,...,"POLYGON((725.4 319.0356,843.708 321.9564,800.9...",None,0,0,None,None,None,None,None,0
9,SOL2010-06-13T09:39:26L103C069,2010-06-13 09:39:26,2010-06-13 09:56:26,2010-06-13T09:45:38,"POLYGON((768 307.2,844.8 307.2,844.8 384,768 3...",POINT(806.4 345.6),877.337062,806.4,345.6,1,...,"POLYGON((725.4 319.0356,843.708 321.9564,800.9...",None,0,0,None,None,None,None,None,0
